## 1. 지식기반 데이터를 임베딩하여 벡터DB에 저장 후 Retriever 예졔

- **실제 Google Gemini(Google Generative AI) 생태계 기반으로**
  - 임베딩 생성
  - 벡터 DB(FAISS) 검색
  - LLM 응답 생성
  을 **통합 파이프라인**으로 구현하는 방법을 실습한다.
- RAG(Retrieval-Augmented Generation)의 전체 흐름을  
  최신 오픈소스와 Google의 공식 AI API로 직접 체험할 수 있도록 한다.

---

## 2. 코드 핵심 내용 및 주요 포인트

### 2.1 Gemini 임베딩과 벡터스토어 생성

- `GoogleGenerativeAIEmbeddings`를 이용해 텍스트(문서/지식베이스)를 **의미 벡터**로 변환
- `FAISS` 벡터스토어에 저장하여 **유사도 검색**이 가능하도록 구성

### 2.2 RAG 프롬프트 템플릿

- 검색 결과로 추출된 문서(컨텍스트)와 사용자의 질문을
  하나의 템플릿으로 LLM에 전달

### 2.3 Gemini LLM 연동

- `ChatGoogleGenerativeAI`를 통해 Gemini 최신 LLM을 사용  
  (Gemini Flash, Pro 등)
- 프롬프트에 기반한 **정확하고 맥락 반영된 답변 생성**

### 2.4 LCEL 기반 체인 자동화

- **Retriever → 컨텍스트 포맷팅 → 프롬프트 → LLM → 출력 파싱**을  
  파이프라인(LCEL) 한 줄로 연결
- 복잡한 RAG 작업을 코드 최소화로 자동화

---

## 3. 실무적 교육 메시지

- **RAG(검색 증강 생성) 파이프라인의 핵심 원리**:  
  검색 + 생성이 결합된 고도화된 AI 워크플로우의 표준
- **Gemini 활용법 실습**:  
  구글 공식 임베딩/LLM API와 최신 LangChain 활용 실전 노하우
- **확장성/자동화**:  
  실무에서 대규모 지식베이스, 엔터프라이즈 챗봇, 문서 Q&A 등에  
  즉시 응용 가능한 기술 구조를 익힘


In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 환경변수 로드
load_dotenv()

# 지식베이스 문서들
knowledge_base = [
    "LangChain은 2022년에 Harrison Chase가 개발한 프레임워크입니다.",
    "LangChain의 주요 구성요소는 LLM, 프롬프트, 메모리, 체인, 에이전트입니다.",
    "체인은 여러 구성요소를 연결하여 복잡한 작업을 수행합니다.",
    "에이전트는 도구를 사용하여 동적으로 작업을 수행할 수 있습니다."
]

# Gemini 임베딩 벡터 저장소 생성
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)
vectorstore = FAISS.from_texts(knowledge_base, embeddings)

In [ ]:
retriever = vectorstore.as_retriever()

## retriever = vectorstore.as_retriever()의 의미

### 1. 정의 및 목적

- **vectorstore.as_retriever()**는  
  벡터DB(예: FAISS)에 저장된 임베딩 기반 문서들을  
  **"검색기(Retriever)" 객체로 변환**하는 함수입니다.
- Retriever는 **입력 쿼리(질문 등)**를 받아  
  **의미적으로 가장 유사한 문서들을 자동으로 찾아 반환**하는 역할을 합니다.

### 2. 왜 필요한가?

- **RAG(검색 증강 생성) 워크플로우**에서  
  LLM에게 컨텍스트(문맥)을 제공하기 위해  
  반드시 "질문 → 문서 검색 → 문서 전달" 절차가 필요합니다.
- `as_retriever()`를 통해  
  벡터DB를 LLM/RAG 파이프라인과 바로 연결할 수 있습니다.

### 3. 실무적 역할

- **vectorstore(FAISS 등)는**  
  문서 저장 및 벡터 검색 API를 모두 포함
- **Retriever로 변환하면:**  
  - LangChain 표준 체인(LCEL) 및 RAG 파이프라인에 **직접 연결** 가능
  - 입력 쿼리를 받아 자동으로 유사도 검색을 수행

### 4. 활용 예시

```python
retriever = vectorstore.as_retriever()
top_docs = retriever.invoke("질문 또는 쿼리")

In [ ]:
# RAG 프롬프트 템플릿
template = """다음 컨텍스트를 기반으로 질문에 답해주세요:

{context}

질문: {question}
답변:"""
prompt = ChatPromptTemplate.from_template(template)

# Gemini LLM 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",   # 또는 gemini-pro 등 최신 모델명 사용 가능
    temperature=0.3
)

# RAG 체인 구성 (LCEL)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문하기
question = "LangChain의 주요 구성요소는 무엇인가요?"
answer = rag_chain.invoke(question)
print(f"질문: {question}")
print(f"답변: {answer}")

# 추가 질문
question2 = "체인은 무엇을 하나요?"
answer2 = rag_chain.invoke(question2)
print(f"\n질문: {question2}")
print(f"답변: {answer2}")

### `format_docs(docs)` 함수의 의미

---

### 1. 함수 목적

- **복수 개의 문서(청크)**를  
  하나의 **긴 텍스트(문자열)**로 합치는 역할을 한다.
- 주로 **RAG(검색 증강 생성) 파이프라인에서,**  
  검색된 여러 문서의 내용을 LLM 프롬프트에  
  "컨텍스트"로 전달할 때 사용한다.

---

### 2. 동작 구조

```python
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    